In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!pip install preprocessor
!pip install tweet-preprocessor
!pip install beautifulsoup4
!pip install fasttext
!pip install emoji
import nltk
nltk.download('stopwords')

  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4477 sha256=8ef891838e347d220b9d16a82357892875a0036c6d726f7c7de72adc86d20b70
  Stored in directory: /root/.cache/pip/wheels/0e/b7/36/aa37256db62b4bfd35a6f1b5536e9ba843f257b79dcbf3d5f1
Successfully built preprocessor
     |████████████████████████████████| 68 kB 5.7 MB/s 
  Using cached pybind11-2.8.1-py2.py3-none-any.whl (208 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3126223 sha256=39e5aaf7e5ff27e7b498bd14a511ce0efa9fc30651ca4690263bd07bcbc8a8f3
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
     |████████████████████████████████| 170 kB 13.1 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=236678e5e6bdbbf9991f77dfef06aefd457e3d2caaaa353cc07ec2f35e54fca7
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427b

True

In [3]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import preprocessor
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import fasttext
import csv
import datetime
from bs4 import BeautifulSoup
import re
import itertools
import emoji

%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import json
import argparse
import importlib
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from sklearn import metrics

def getData(type):

    folder_path = "/content/drive/MyDrive/CS640-project/DataTwitter/"+ type + "/"
    dfs = []
    for filename in os.listdir(folder_path):
        if filename.endswith('txt'):
            path = os.path.join(folder_path, filename)
            df = pd.read_csv(path, sep="\\t", header=None)

            df.rename(columns={1: "tweet", 2: "sentiment"}, inplace=True)
            df = df.drop(columns=[df.columns[0], df.columns[3]])
            df['sentiment'] = df['sentiment'].replace(['anger'],0)
            df['sentiment'] = df['sentiment'].replace(['fear'],1)
            df['sentiment'] = df['sentiment'].replace(['joy'],2)
            df['sentiment'] = df['sentiment'].replace(['sadness'],3)

            dfs.append(df)


    df = pd.concat(dfs)
    # df.columns = ['text', 'label']
    # df['label_numeric'] = df['label'].astype('category').cat.codes
    print('total ' + type +' samples : ', len(df))
    print(df['sentiment'].value_counts())

    return df




train = getData("Train")
dev = getData("Dev")
test = getData("Test")
# train = pd.concat([train, test])
# train = train.sample(frac=0.01, replace=True, random_state=1)
# # test = test.sample(frac=0.1, replace=True, random_state=1)
X_train = train.filter(regex='tweet').tweet.values
y_train = train.filter(regex='sentiment').sentiment.values
X_dev = dev.filter(regex='tweet').tweet.values
y_dev = dev.filter(regex='sentiment').sentiment.values
X_test = test.filter(regex='tweet').tweet.values
y_test = test.filter(regex='sentiment').sentiment.values



class_mapping = ['anger', 'fear', 'joy', 'sadness']
print(class_mapping)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


total Train samples :  3613
1    1147
0     857
2     823
3     786
Name: sentiment, dtype: int64
total Dev samples :  347
1    110
0     84
2     79
3     74
Name: sentiment, dtype: int64
total Test samples :  3142
1    995
0    760
2    714
3    673
Name: sentiment, dtype: int64
['anger', 'fear', 'joy', 'sadness']


In [7]:
import os
import json
import argparse
import importlib
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from sklearn import metrics

def getData():

    folder_path = "/content/drive/MyDrive/CS640-project/DataInstagram/"
    dfs = []
    for filename in os.listdir(folder_path):
        if filename.endswith('csv'):
            path = os.path.join(folder_path, filename)
            df = pd.read_csv(path)

            df.rename(columns={'imagename': "image_id", "Contents": "tweet", 'Q5_presence_asian': "isEastAsian"}, inplace=True)
            df = df.drop(columns=[df.columns[1], df.columns[2], df.columns[3], df.columns[4], df.columns[6], df.columns[7], df.columns[8], df.columns[9],
                                  df.columns[10], df.columns[11], df.columns[13], df.columns[14], df.columns[15], df.columns[16], df.columns[17],
                                  df.columns[18], df.columns[19], df.columns[20], df.columns[21], df.columns[22], df.columns[23], df.columns[24], df.columns[25]])
    
            # df['sentiment'] = df['sentiment'].replace(['anger'],0)
            # df['sentiment'] = df['sentiment'].replace(['fear'],1)
            # df['sentiment'] = df['sentiment'].replace(['joy'],2)
            # df['sentiment'] = df['sentiment'].replace(['sadness'],3)

            dfs.append(df)


    df = pd.concat(dfs)
    # df.columns = ['text', 'label']
    # df['label_numeric'] = df['label'].astype('category').cat.codes
    print('total ' + "insta" +' samples : ', len(df))
    # print(df['isEastAsian'].value_counts())

    return df




Insta_data = getData()
Insta_data = Insta_data.dropna(thresh=2)
X_insta = Insta_data.tweet.values

# print(Insta_train)
print(Insta_data)

# train = pd.concat([train, test])
# train = train.sample(frac=0.01, replace=True, random_state=1)
# # test = test.sample(frac=0.1, replace=True, random_state=1)
# X_insta = train.filter(regex='tweet').tweet.values
# y_train = train.filter(regex='sentiment').sentiment.values








total insta samples :  9648
         image_id  ...                                    translated_text
0     B-A5FIIIEKJ  ...  #covid #covid2020 #covidvirus #virus #coronava...
1     B-A74YQn_4a  ...  Well this is the final mural of my trip in Aus...
2     B-AajnDp6GQ  ...  We have arrived!!! Keep following the movement...
3     B-AarR4pUQA  ...                                              😻😻😻😻😻
4     B-AE1y3HD-Z  ...  AT MY HOME 🏡_x000D__x000D__x000D__x000D_\n# st...
...           ...  ...                                                ...
9643  CEZyheBgtMd  ...  #water #foryou #followforfollowback #photograp...
9644  CEZYoyRluKG  ...  #like4likes #20likes #tagforlikes #instalikes ...
9645  CEZz-solaUF  ...  🙈🥰😍👉🏽 @love_serie_karma #daancorona #daancoron...
9646  CEZZ4P4j3rh  ...  💥🚨 𝗙𝗔𝗟𝗟 𝗜𝗦 𝗖𝗢𝗠𝗜𝗡𝗚 🚨💥⁣_x000D__x000D__x000D__x00...
9647  CEZZyqshFLq  ...  Follow me @kestine_kylie _x000D__x000D__x000D_...

[9648 rows x 5 columns]


In [8]:
# def text_preprocessing(text):  
#     text = re.sub(r'(@.*?)[\s]', '', text)
#     text = re.sub(r'&amp;', '&', text)
#     text = re.sub(r"<[\w'/'\s]*>",'' ,text)
#     text = re.sub(r"[\'\"\-]+", '',text)
#     text = re.sub(r"@[\w]+", '',text)
#     text = re.sub(r'http\S+', '',text)
#     text = re.sub(r'\s+', '', text).strip()
#     return text
preprocessor.set_options(preprocessor.OPT.URL,preprocessor.OPT.RESERVED)
stop_words = nltk.corpus.stopwords.words('english')
def text_preprocessing(text):   
    text = preprocessor.tokenize(text)
    text = ' '.join([word for word in text.split(' ') if word.lower() not in stop_words])
    return text

In [9]:
lemmatizer = WordNetLemmatizer()

def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }


def load_dict_contractions_slangs():
    
    cont = {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "i'd":"i would",
        "i'll":"i will",
        "i'm":"i am",
        "i'm'a":"i am about to",
        "i'm'o":"i am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "i've":"i have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks",
        "rn":"right now",
        "atm": "at the moment",
        "idk": "i dont know",
        "cuz": "because",
        "bcuz": "because",
        "ur": "you are",
        "ly": "love you",
        "lol": "laugh out loud",
        "rofl": "rolling on floor laughing",
        "lmao": "laughing my ass off",
        "ok": "okay",
        "ty": "thank you",
        "fav": "favorite",
        "omg": "oh my god"
        }
    
    to_ret = {}
    for k,v in cont.items():
        key = k.lower().replace("'",'')
        to_ret[key] = v.lower()
        
    return to_ret


def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        #Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)


def tweet_cleaning_for_sentiment_analysis(tweet):    
    
    #Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
    #Special case not handled previously.
    tweet = tweet.replace('\x92',"'")
    #Removal of account
    tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)", " ", tweet).split())
    # removal of hashtag
    tweet = tweet.replace('#','')
    #Removal of address
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    #Lower case
    tweet = tweet.lower()
    # rempval of 'RT'
    tweet = tweet.replace('rt','')
    # Removal of Punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    # removal of stop words
    tweet = ' '.join([word for word in tweet.split(' ') if word not in stop_words])
    # CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS_SLANGS = load_dict_contractions_slangs()
    tweet = tweet.replace("’","'").replace("'","")
    words = tweet.split()
    reformed = [CONTRACTIONS_SLANGS[word] if word in CONTRACTIONS_SLANGS else word for word in words]
    tweet = " ".join(reformed)

    # Standardizing words - lemmatization
    tweet = ' '.join([lemmatizer.lemmatize(word) for word in tweet.split(' ')]).lower()
    
    #Deal with smileys
    #source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    # replace emojis with desctiption - remove accents - remove underscores
    tweet = emoji.demojize(tweet).lower()
    tweet = strip_accents(tweet).lower()
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split()).replace('_',' ')
    
    # remove new line characters
    tweet = tweet.strip().replace('\\n',' ')
    
    # only keep alphabets
    tweet = ''.join([c for c in tweet if (c.isalpha() or c ==' ')]).strip()
    
    # rempove double spaces and triple spaces
    tweet = tweet.replace('   ',' ').replace('  ',' ')
    
    # one word replacements
    replacement_dict = {
        'u': 'you',
        'v': 'we',
        'r': 'are',
        'w': 'we',
        'n': 'and',
        'nd': 'and',
        '&': 'and'
    }
    
    tweet = ' '.join([replacement_dict[word] if word in replacement_dict else word for word in tweet.split(' ')])
    
    # removal of stop words
    tweet = ' '.join([word for word in tweet.split(' ') if word not in stop_words])

    return tweet
    

In [10]:
def preprocessAllData(X):
  count = []
  res = []
  for data in X:
    if data != "":
      try:
        res.append(tweet_cleaning_for_sentiment_analysis(data))
        count.append(len(res))
      except:
        #nan values are filtered here 
        pass
  print(count)
  return np.array(res)

# X_train = tweet_cleaning_for_sentiment_analysis(X_train)
# X_dev = preprocessAllData(X_dev)
# X_test = preprocessAllData(X_test)
# X_insta = preprocessAllData(Insta_train)

train['tweet'] = train['tweet'].apply(tweet_cleaning_for_sentiment_analysis)
dev['tweet'] = dev['tweet'].apply(tweet_cleaning_for_sentiment_analysis)
test['tweet'] = test['tweet'].apply(tweet_cleaning_for_sentiment_analysis)
X_train = train.filter(regex='tweet').tweet.values
y_train = train.filter(regex='sentiment').sentiment.values
X_dev = dev.filter(regex='tweet').tweet.values
y_dev = dev.filter(regex='sentiment').sentiment.values
X_test = test.filter(regex='tweet').tweet.values
y_test = test.filter(regex='sentiment').sentiment.values



Naive Bayes 

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
num_features = 3000
#count vectorizer 
cv = CountVectorizer(max_features=num_features)
cv.fit_transform(X_train)
X_train_cv = cv.transform(X_train)
X_dev_cv = cv.transform(X_dev)
X_test_cv = cv.transform(X_test)

#tfidf vectorizer 

tfidf = TfidfVectorizer(max_features=num_features)
tfidf.fit_transform(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_dev_tfidf = tfidf.transform(X_dev)
X_test_tfidf = tfidf.transform(X_test)


#word2vec

# word2vecModel = Word2Vec(X_train, min_count=1,size= 50,workers=3, window =3, sg = 1)
# X_train_word2vec = []
# for i in X_train:
#   X_train_word2vec.append(word2vecModel[i])
# print(X_train_word2vec)



In [52]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
print("Naive Bayes:")
model_cv = MultinomialNB()
model_cv.fit(X_train_cv, y_train)
cv_nb_score = model_cv.score(X_dev_cv, y_dev)
Y_pred = model_cv.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred, target_names = class_mapping))
print("count vectorizer score: ", cv_nb_score)
print("instagram data results: ")
InstaPredictions = []

naiveBayesInstaPredictions = []
Insta_ea = Insta_data.isEastAsian.values
eastAsian = []
index = 0
for row in X_insta:
  try: 
    emotion_pred = class_mapping[model_cv.predict(cv.transform([row]))[0]]
    naiveBayesInstaPredictions.append(emotion_pred)
    eastAsian.append(Insta_ea[index])
  except:
    pass
  index += 1
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())
with open('Insta_predictions_cv_nb.csv', 'w') as csvfile:
  writer=csv.writer(csvfile, delimiter=',')
  writer.writerows(zip(X_insta, naiveBayesInstaPredictions))
# print(pd.DataFrame(eastAsian).value_counts())
eastAsianFear = [0, 0]
eastAsianAnger = [0, 0]

for person in range(len(eastAsian)):
  if eastAsian[person] == 1:
    if naiveBayesInstaPredictions[person] == 'fear':
      eastAsianFear[0] += 1
    if naiveBayesInstaPredictions[person] == 'anger':
      eastAsianAnger[0] += 1
    else:
      eastAsianFear[1] += 1
      eastAsianAnger[1] += 1
# print(eastAsianFear, eastAsianAnger)
from scipy.stats import chi2_contingency 
# using Pearson’s chi-squared statistic
# corrected for the Yates’ continuity
observed = np.array([eastAsianFear, eastAsianAnger])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
print("correlation vals")
print("Pearson chi square value: ", chi_val, "p-value:" , p_val)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)
tfidf_nb_score = model_tfidf.score(X_dev_tfidf, y_dev)
Y_pred = model_tfidf.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred, target_names = class_mapping))
print("tfidf vectorizer score: ", tfidf_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
Insta_ea = Insta_data.isEastAsian.values
eastAsian = []
index = 0
for row in X_insta:
  try: 
    emotion_pred = class_mapping[model_tfidf.predict(tfidf.transform([row]))[0]]
    naiveBayesInstaPredictions.append(emotion_pred)
    eastAsian.append(Insta_ea[index])
  except:
    pass
  index += 1
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())

with open('Insta_predictions_tfidf_nb.csv', 'w') as csvfile:
  writer=csv.writer(csvfile, delimiter=',')
  writer.writerows(zip(X_insta, naiveBayesInstaPredictions))

eastAsianFear = [0, 0]
eastAsianAnger = [0, 0]
for person in range(len(eastAsian)):
  if eastAsian[person] == 1:
    if naiveBayesInstaPredictions[person] == 'fear':
      eastAsianFear[0] += 1
    if naiveBayesInstaPredictions[person] == 'anger':
      eastAsianAnger[0] += 1
    else:
      eastAsianFear[1] += 1
      eastAsianAnger[1] += 1
# print(eastAsianFear, eastAsianAnger)
observed = np.array([eastAsianFear, eastAsianAnger])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
print("correlation vals")
print("Pearson chi square value: ", chi_val, "p-value:" , p_val)


Naive Bayes:
              precision    recall  f1-score   support

       anger       0.78      0.75      0.76        84
        fear       0.81      0.72      0.76       110
         joy       0.77      0.82      0.80        79
     sadness       0.75      0.85      0.80        74

    accuracy                           0.78       347
   macro avg       0.78      0.79      0.78       347
weighted avg       0.78      0.78      0.78       347

count vectorizer score:  0.7780979827089337
instagram data results: 
fear       4058
joy        2111
anger      1712
sadness    1708
dtype: int64
correlation vals
Pearson chi square value:  50.98779180434007 p-value: 9.29422273346263e-13
              precision    recall  f1-score   support

       anger       0.83      0.77      0.80        84
        fear       0.77      0.78      0.77       110
         joy       0.77      0.80      0.78        79
     sadness       0.80      0.81      0.81        74

    accuracy                           0.7

9648 9589


In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
print("Logistic Regression:")
model_cv = LogisticRegression()
model_cv.fit(X_train_cv, y_train)
cv_nb_score = model_cv.score(X_dev_cv, y_dev)
Y_pred = model_cv.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred, target_names = class_mapping))
print("count vectorizer score: ", cv_nb_score)
print("instagram data results: ")
InstaPredictions = []

naiveBayesInstaPredictions = []
Insta_ea = Insta_data.isEastAsian.values
eastAsian = []
index = 0
for row in X_insta:
  try: 
    emotion_pred = class_mapping[model_cv.predict(cv.transform([row]))[0]]
    naiveBayesInstaPredictions.append(emotion_pred)
    eastAsian.append(Insta_ea[index])
  except:
    pass
  index += 1
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())
with open('Insta_predictions_cv_logistic_regression.csv', 'w') as csvfile:
  writer=csv.writer(csvfile, delimiter=',')
  writer.writerows(zip(X_insta, naiveBayesInstaPredictions))
# print(pd.DataFrame(eastAsian).value_counts())
eastAsianFear = [0, 0]
eastAsianAnger = [0, 0]

for person in range(len(eastAsian)):
  if eastAsian[person] == 1:
    if naiveBayesInstaPredictions[person] == 'fear':
      eastAsianFear[0] += 1
    if naiveBayesInstaPredictions[person] == 'anger':
      eastAsianAnger[0] += 1
    else:
      eastAsianFear[1] += 1
      eastAsianAnger[1] += 1
# print(eastAsianFear, eastAsianAnger)
from scipy.stats import chi2_contingency 
# using Pearson’s chi-squared statistic
# corrected for the Yates’ continuity
observed = np.array([eastAsianFear, eastAsianAnger])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
print("correlation vals")
print("Pearson chi square value: ", chi_val, "p-value:" , p_val)

model_tfidf = LogisticRegression()
model_tfidf.fit(X_train_tfidf, y_train)
tfidf_nb_score = model_tfidf.score(X_dev_tfidf, y_dev)
Y_pred = model_tfidf.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred, target_names = class_mapping))
print("tfidf vectorizer score: ", tfidf_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
Insta_ea = Insta_data.isEastAsian.values
eastAsian = []
index = 0
for row in X_insta:
  try: 
    emotion_pred = class_mapping[model_tfidf.predict(tfidf.transform([row]))[0]]
    naiveBayesInstaPredictions.append(emotion_pred)
    eastAsian.append(Insta_ea[index])
  except:
    pass
  index += 1
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())

with open('Insta_predictions_tfidf_logistic_regression.csv', 'w') as csvfile:
  writer=csv.writer(csvfile, delimiter=',')
  writer.writerows(zip(X_insta, naiveBayesInstaPredictions))

eastAsianFear = [0, 0]
eastAsianAnger = [0, 0]
for person in range(len(eastAsian)):
  if eastAsian[person] == 1:
    if naiveBayesInstaPredictions[person] == 'fear':
      eastAsianFear[0] += 1
    if naiveBayesInstaPredictions[person] == 'anger':
      eastAsianAnger[0] += 1
    else:
      eastAsianFear[1] += 1
      eastAsianAnger[1] += 1
# print(eastAsianFear, eastAsianAnger)
observed = np.array([eastAsianFear, eastAsianAnger])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
print("correlation vals")
print("Pearson chi square value: ", chi_val, "p-value:" , p_val)


Logistic Regression:
              precision    recall  f1-score   support

       anger       0.84      0.83      0.84        84
        fear       0.81      0.85      0.83       110
         joy       0.87      0.85      0.86        79
     sadness       0.83      0.81      0.82        74

    accuracy                           0.84       347
   macro avg       0.84      0.83      0.84       347
weighted avg       0.84      0.84      0.84       347

count vectorizer score:  0.8357348703170029
instagram data results: 
fear       5479
anger      1663
joy        1578
sadness     869
dtype: int64
correlation vals
Pearson chi square value:  77.66328091123472 p-value: 1.2219192901169517e-18
              precision    recall  f1-score   support

       anger       0.86      0.80      0.83        84
        fear       0.82      0.80      0.81       110
         joy       0.84      0.84      0.84        79
     sadness       0.80      0.89      0.84        74

    accuracy                    

In [51]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
print("Linear SVC:")
model_cv = LinearSVC()
model_cv.fit(X_train_cv, y_train)
cv_nb_score = model_cv.score(X_dev_cv, y_dev)
Y_pred = model_cv.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred, target_names = class_mapping))
print("count vectorizer score: ", cv_nb_score)
print("instagram data results: ")
InstaPredictions = []

naiveBayesInstaPredictions = []
Insta_ea = Insta_data.isEastAsian.values
eastAsian = []
index = 0
for row in X_insta:
  try: 
    emotion_pred = class_mapping[model_cv.predict(cv.transform([row]))[0]]
    naiveBayesInstaPredictions.append(emotion_pred)
    eastAsian.append(Insta_ea[index])
  except:
    pass
  index += 1
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())
with open('Insta_predictions_cv_LinearSVC.csv', 'w') as csvfile:
  writer=csv.writer(csvfile, delimiter=',')
  writer.writerows(zip(X_insta, naiveBayesInstaPredictions))
# print(pd.DataFrame(eastAsian).value_counts())
eastAsianFear = [0, 0]
eastAsianAnger = [0, 0]

for person in range(len(eastAsian)):
  if eastAsian[person] == 1:
    if naiveBayesInstaPredictions[person] == 'fear':
      eastAsianFear[0] += 1
    if naiveBayesInstaPredictions[person] == 'anger':
      eastAsianAnger[0] += 1
    else:
      eastAsianFear[1] += 1
      eastAsianAnger[1] += 1
# print(eastAsianFear, eastAsianAnger)
from scipy.stats import chi2_contingency 
# using Pearson’s chi-squared statistic
# corrected for the Yates’ continuity
observed = np.array([eastAsianFear, eastAsianAnger])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
print("correlation vals")
print("Pearson chi square value: ", chi_val, "p-value:" , p_val)

model_tfidf = LinearSVC()
model_tfidf.fit(X_train_tfidf, y_train)
tfidf_nb_score = model_tfidf.score(X_dev_tfidf, y_dev)
Y_pred = model_tfidf.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred, target_names = class_mapping))
print("tfidf vectorizer score: ", tfidf_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
Insta_ea = Insta_data.isEastAsian.values
eastAsian = []
index = 0
for row in X_insta:
  try: 
    emotion_pred = class_mapping[model_tfidf.predict(tfidf.transform([row]))[0]]
    naiveBayesInstaPredictions.append(emotion_pred)
    eastAsian.append(Insta_ea[index])
  except:
    pass
  index += 1
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())

with open('Insta_predictions_tfidf_linearSVC.csv', 'w') as csvfile:
  writer=csv.writer(csvfile, delimiter=',')
  writer.writerows(zip(X_insta, naiveBayesInstaPredictions))

eastAsianFear = [0, 0]
eastAsianAnger = [0, 0]
for person in range(len(eastAsian)):
  if eastAsian[person] == 1:
    if naiveBayesInstaPredictions[person] == 'fear':
      eastAsianFear[0] += 1
    if naiveBayesInstaPredictions[person] == 'anger':
      eastAsianAnger[0] += 1
    else:
      eastAsianFear[1] += 1
      eastAsianAnger[1] += 1
# print(eastAsianFear, eastAsianAnger)
observed = np.array([eastAsianFear, eastAsianAnger])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
print("correlation vals")
print("Pearson chi square value: ", chi_val, "p-value:" , p_val)


Linear SVC:
              precision    recall  f1-score   support

       anger       0.85      0.80      0.82        84
        fear       0.79      0.85      0.82       110
         joy       0.84      0.81      0.83        79
     sadness       0.81      0.80      0.80        74

    accuracy                           0.82       347
   macro avg       0.82      0.81      0.82       347
weighted avg       0.82      0.82      0.82       347

count vectorizer score:  0.8184438040345822
instagram data results: 
fear       5222
joy        1665
anger      1663
sadness    1039
dtype: int64
correlation vals
Pearson chi square value:  72.40528852928797 p-value: 1.7524369800453984e-17
              precision    recall  f1-score   support

       anger       0.88      0.81      0.84        84
        fear       0.84      0.79      0.82       110
         joy       0.82      0.86      0.84        79
     sadness       0.76      0.86      0.81        74

    accuracy                           0.

In [54]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
print("MLP Classifier:")
model_cv = MLPClassifier()
model_cv.fit(X_train_cv, y_train)
cv_nb_score = model_cv.score(X_dev_cv, y_dev)
Y_pred = model_cv.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred, target_names = class_mapping))
print("count vectorizer score: ", cv_nb_score)
print("instagram data results: ")
InstaPredictions = []

naiveBayesInstaPredictions = []
Insta_ea = Insta_data.isEastAsian.values
eastAsian = []
index = 0
for row in X_insta:
  try: 
    emotion_pred = class_mapping[model_cv.predict(cv.transform([row]))[0]]
    naiveBayesInstaPredictions.append(emotion_pred)
    eastAsian.append(Insta_ea[index])
  except:
    pass
  index += 1
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())
with open('Insta_predictions_cv_MLP_classifier.csv', 'w') as csvfile:
  writer=csv.writer(csvfile, delimiter=',')
  writer.writerows(zip(X_insta, naiveBayesInstaPredictions))
# print(pd.DataFrame(eastAsian).value_counts())
eastAsianFear = [0, 0]
eastAsianAnger = [0, 0]

for person in range(len(eastAsian)):
  if eastAsian[person] == 1:
    if naiveBayesInstaPredictions[person] == 'fear':
      eastAsianFear[0] += 1
    if naiveBayesInstaPredictions[person] == 'anger':
      eastAsianAnger[0] += 1
    else:
      eastAsianFear[1] += 1
      eastAsianAnger[1] += 1
# print(eastAsianFear, eastAsianAnger)
from scipy.stats import chi2_contingency 
# using Pearson’s chi-squared statistic
# corrected for the Yates’ continuity
observed = np.array([eastAsianFear, eastAsianAnger])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
print("correlation vals")
print("Pearson chi square value: ", chi_val, "p-value:" , p_val)

model_tfidf = MLPClassifier()
model_tfidf.fit(X_train_tfidf, y_train)
tfidf_nb_score = model_tfidf.score(X_dev_tfidf, y_dev)
Y_pred = model_tfidf.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred, target_names = class_mapping))
print("tfidf vectorizer score: ", tfidf_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
Insta_ea = Insta_data.isEastAsian.values
eastAsian = []
index = 0
for row in X_insta:
  try: 
    emotion_pred = class_mapping[model_tfidf.predict(tfidf.transform([row]))[0]]
    naiveBayesInstaPredictions.append(emotion_pred)
    eastAsian.append(Insta_ea[index])
  except:
    pass
  index += 1
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())

with open('Insta_predictions_tfidf_MLP_classifier.csv', 'w') as csvfile:
  writer=csv.writer(csvfile, delimiter=',')
  writer.writerows(zip(X_insta, naiveBayesInstaPredictions))

eastAsianFear = [0, 0]
eastAsianAnger = [0, 0]
for person in range(len(eastAsian)):
  if eastAsian[person] == 1:
    if naiveBayesInstaPredictions[person] == 'fear':
      eastAsianFear[0] += 1
    if naiveBayesInstaPredictions[person] == 'anger':
      eastAsianAnger[0] += 1
    else:
      eastAsianFear[1] += 1
      eastAsianAnger[1] += 1
# print(eastAsianFear, eastAsianAnger)
observed = np.array([eastAsianFear, eastAsianAnger])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
print("correlation vals")
print("Pearson chi square value: ", chi_val, "p-value:" , p_val)


MLP Classifier:
              precision    recall  f1-score   support

       anger       0.84      0.82      0.83        84
        fear       0.82      0.83      0.82       110
         joy       0.82      0.86      0.84        79
     sadness       0.79      0.76      0.77        74

    accuracy                           0.82       347
   macro avg       0.82      0.82      0.82       347
weighted avg       0.82      0.82      0.82       347

count vectorizer score:  0.8184438040345822
instagram data results: 
fear       4564
anger      1882
joy        1650
sadness    1493
dtype: int64
correlation vals
Pearson chi square value:  20.896849124797836 p-value: 4.846912120902017e-06
              precision    recall  f1-score   support

       anger       0.84      0.81      0.82        84
        fear       0.84      0.77      0.81       110
         joy       0.79      0.89      0.83        79
     sadness       0.76      0.78      0.77        74

    accuracy                         

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
print("Random Forest Classifier:")
model_cv = RandomForestClassifier()
model_cv.fit(X_train_cv, y_train)
cv_nb_score = model_cv.score(X_dev_cv, y_dev)
Y_pred = model_cv.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred, target_names = class_mapping))
print("count vectorizer score: ", cv_nb_score)
print("instagram data results: ")
InstaPredictions = []

naiveBayesInstaPredictions = []
Insta_ea = Insta_data.isEastAsian.values
eastAsian = []
index = 0
for row in X_insta:
  try: 
    emotion_pred = class_mapping[model_cv.predict(cv.transform([row]))[0]]
    naiveBayesInstaPredictions.append(emotion_pred)
    eastAsian.append(Insta_ea[index])
  except:
    pass
  index += 1
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())
with open('Insta_predictions_cv_RandomForest.csv', 'w') as csvfile:
  writer=csv.writer(csvfile, delimiter=',')
  writer.writerows(zip(X_insta, naiveBayesInstaPredictions))
# print(pd.DataFrame(eastAsian).value_counts())
eastAsianFear = [0, 0]
eastAsianAnger = [0, 0]

for person in range(len(eastAsian)):
  if eastAsian[person] == 1:
    if naiveBayesInstaPredictions[person] == 'fear':
      eastAsianFear[0] += 1
    if naiveBayesInstaPredictions[person] == 'anger':
      eastAsianAnger[0] += 1
    else:
      eastAsianFear[1] += 1
      eastAsianAnger[1] += 1
# print(eastAsianFear, eastAsianAnger)
from scipy.stats import chi2_contingency 
# using Pearson’s chi-squared statistic
# corrected for the Yates’ continuity
observed = np.array([eastAsianFear, eastAsianAnger])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
print("correlation vals")
print("Pearson chi square value: ", chi_val, "p-value:" , p_val)

model_tfidf = RandomForestClassifier()
model_tfidf.fit(X_train_tfidf, y_train)
tfidf_nb_score = model_tfidf.score(X_dev_tfidf, y_dev)
Y_pred = model_tfidf.predict(X_dev_cv)
print(classification_report(y_dev,Y_pred, target_names = class_mapping))
print("tfidf vectorizer score: ", tfidf_nb_score)
print("instagram data results: ")
naiveBayesInstaPredictions = []
Insta_ea = Insta_data.isEastAsian.values
eastAsian = []
index = 0
for row in X_insta:
  try: 
    emotion_pred = class_mapping[model_tfidf.predict(tfidf.transform([row]))[0]]
    naiveBayesInstaPredictions.append(emotion_pred)
    eastAsian.append(Insta_ea[index])
  except:
    pass
  index += 1
print(pd.DataFrame(naiveBayesInstaPredictions).value_counts())

with open('Insta_predictions_tfidf_RandomForest.csv', 'w') as csvfile:
  writer=csv.writer(csvfile, delimiter=',')
  writer.writerows(zip(X_insta, naiveBayesInstaPredictions))

eastAsianFear = [0, 0]
eastAsianAnger = [0, 0]
for person in range(len(eastAsian)):
  if eastAsian[person] == 1:
    if naiveBayesInstaPredictions[person] == 'fear':
      eastAsianFear[0] += 1
    if naiveBayesInstaPredictions[person] == 'anger':
      eastAsianAnger[0] += 1
    else:
      eastAsianFear[1] += 1
      eastAsianAnger[1] += 1
# print(eastAsianFear, eastAsianAnger)
observed = np.array([eastAsianFear, eastAsianAnger])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
print("correlation vals")
print("Pearson chi square value: ", chi_val, "p-value:" , p_val)


Random Forest Classifier:
              precision    recall  f1-score   support

       anger       0.91      0.87      0.89        84
        fear       0.88      0.84      0.86       110
         joy       0.87      0.86      0.87        79
     sadness       0.71      0.81      0.76        74

    accuracy                           0.84       347
   macro avg       0.84      0.84      0.84       347
weighted avg       0.85      0.84      0.85       347

count vectorizer score:  0.8443804034582133
instagram data results: 
sadness    5386
fear       2077
anger      1122
joy        1004
dtype: int64
correlation vals
Pearson chi square value:  17.52638454458093 p-value: 2.8334819510252926e-05
              precision    recall  f1-score   support

       anger       0.84      0.85      0.84        84
        fear       0.90      0.82      0.86       110
         joy       0.87      0.86      0.87        79
     sadness       0.74      0.84      0.78        74

    accuracy               

In [17]:
import numpy as np
from scipy.stats import chi2_contingency 
# using Pearson’s chi-squared statistic
# corrected for the Yates’ continuity
observed = np.array([[60, 10], [30, 25]])
chi_val, p_val, dof, expected =  chi2_contingency(observed)
chi_val, p_val, dof, expected

(13.3364898989899, 0.0002602911116400899, 1, array([[50.4, 19.6],
        [39.6, 15.4]]))

In [38]:
for lbl in [0,1,2,3]:
    
    train_0 = train[train['sentiment'] == lbl]
    train_rest = train[train['sentiment'] != lbl]
    train_rest = train_rest.sample(int(len(train_0)))
                                         

    train_sampled = train_0.append(train_rest)
    train_sampled = train_sampled.sample(frac=1).reset_index()

    Y_train = train_sampled['sentiment'].apply(lambda x: 0 if x == lbl else 1 ).tolist()
    Y_test = dev['sentiment'].apply(lambda x: 0 if x == lbl else 1 ).tolist()

    num_features = 3000
    model_tfidf = TfidfVectorizer(max_features=num_features)
    model_tfidf.fit(train['tweet'])

    model_cv = CountVectorizer(max_features=num_features)
    model_cv.fit(train['tweet'])

    X_train_tfidf = model_tfidf.transform(train_sampled['tweet']).toarray()
    X_test_tfidf = model_tfidf.transform(dev['tweet']).toarray()
    X_train_cv = model_cv.transform(train_sampled['tweet']).toarray()
    X_test_cv = model_cv.transform(dev['tweet']).toarray()
    

    model_t = LogisticRegression()
    model_t.fit(X_train_tfidf,Y_train)
    Y_pred_t = model_t.predict(X_test_tfidf)
    model_c = LogisticRegression()
    model_c.fit(X_train_cv,Y_train)
    Y_pred_cv = model_c.predict(X_test_cv)
    print("Individual class accuracies for logistic regression  (tfidf vectiorizer)")
    print(classification_report(Y_test,Y_pred_t,target_names = [class_mapping[lbl],'not-'+class_mapping[lbl]]))
    print("Individual class accuracies for logistic regression  (count vectiorizer)")
    print(classification_report(Y_test,Y_pred_cv,target_names = [class_mapping[lbl],'not-'+class_mapping[lbl]]))

Individual class accuracies for logistic regression  (tfidf vectiorizer)
              precision    recall  f1-score   support

       anger       0.74      0.86      0.80        84
   not-anger       0.95      0.90      0.93       263

    accuracy                           0.89       347
   macro avg       0.85      0.88      0.86       347
weighted avg       0.90      0.89      0.90       347

Individual class accuracies for logistic regression  (count vectiorizer)
              precision    recall  f1-score   support

       anger       0.76      0.88      0.81        84
   not-anger       0.96      0.91      0.93       263

    accuracy                           0.90       347
   macro avg       0.86      0.89      0.87       347
weighted avg       0.91      0.90      0.90       347

Individual class accuracies for logistic regression  (tfidf vectiorizer)
              precision    recall  f1-score   support

        fear       0.76      0.79      0.78       110
    not-fear      

In [41]:
for lbl in [0,1,2,3]:
    
    train_0 = train[train['sentiment'] == lbl]
    train_rest = train[train['sentiment'] != lbl]
    train_rest = train_rest.sample(int(len(train_0)))
                                         

    train_sampled = train_0.append(train_rest)
    train_sampled = train_sampled.sample(frac=1).reset_index()

    Y_train = train_sampled['sentiment'].apply(lambda x: 0 if x == lbl else 1 ).tolist()
    Y_test = dev['sentiment'].apply(lambda x: 0 if x == lbl else 1 ).tolist()

    num_features = 3000
    model_tfidf = TfidfVectorizer(max_features=num_features)
    model_tfidf.fit(train['tweet'])

    model_cv = CountVectorizer(max_features=num_features)
    model_cv.fit(train['tweet'])

    X_train_tfidf = model_tfidf.transform(train_sampled['tweet']).toarray()
    X_test_tfidf = model_tfidf.transform(dev['tweet']).toarray()
    X_train_cv = model_cv.transform(train_sampled['tweet']).toarray()
    X_test_cv = model_cv.transform(dev['tweet']).toarray()
    

    model_t = MultinomialNB()
    model_t.fit(X_train_tfidf,Y_train)
    Y_pred_t = model_t.predict(X_test_tfidf)
    model_c = MultinomialNB()
    model_c.fit(X_train_cv,Y_train)
    Y_pred_cv = model_c.predict(X_test_cv)
    print("Individual class accuracies for Naive Bayes  (tfidf vectiorizer)")
    print(classification_report(Y_test,Y_pred_t,target_names = [class_mapping[lbl],'not-'+class_mapping[lbl]]))
    print("Individual class accuracies for Naive Bayes  (count vectiorizer)")
    print(classification_report(Y_test,Y_pred_cv,target_names = [class_mapping[lbl],'not-'+class_mapping[lbl]]))

Individual class accuracies for Naive Bayes  (tfidf vectiorizer)
              precision    recall  f1-score   support

       anger       0.57      0.88      0.69        84
   not-anger       0.95      0.79      0.86       263

    accuracy                           0.81       347
   macro avg       0.76      0.83      0.78       347
weighted avg       0.86      0.81      0.82       347

Individual class accuracies for Naive Bayes  (count vectiorizer)
              precision    recall  f1-score   support

       anger       0.58      0.88      0.70        84
   not-anger       0.95      0.80      0.87       263

    accuracy                           0.82       347
   macro avg       0.77      0.84      0.79       347
weighted avg       0.86      0.82      0.83       347

Individual class accuracies for Naive Bayes  (tfidf vectiorizer)
              precision    recall  f1-score   support

        fear       0.72      0.81      0.76       110
    not-fear       0.91      0.85      0.8

In [40]:
for lbl in [0,1,2,3]:
    
    train_0 = train[train['sentiment'] == lbl]
    train_rest = train[train['sentiment'] != lbl]
    train_rest = train_rest.sample(int(len(train_0)))
                                         

    train_sampled = train_0.append(train_rest)
    train_sampled = train_sampled.sample(frac=1).reset_index()

    Y_train = train_sampled['sentiment'].apply(lambda x: 0 if x == lbl else 1 ).tolist()
    Y_test = dev['sentiment'].apply(lambda x: 0 if x == lbl else 1 ).tolist()

    num_features = 3000
    model_tfidf = TfidfVectorizer(max_features=num_features)
    model_tfidf.fit(train['tweet'])

    model_cv = CountVectorizer(max_features=num_features)
    model_cv.fit(train['tweet'])

    X_train_tfidf = model_tfidf.transform(train_sampled['tweet']).toarray()
    X_test_tfidf = model_tfidf.transform(dev['tweet']).toarray()
    X_train_cv = model_cv.transform(train_sampled['tweet']).toarray()
    X_test_cv = model_cv.transform(dev['tweet']).toarray()
    

    model_t = LinearSVC()
    model_t.fit(X_train_tfidf,Y_train)
    Y_pred_t = model_t.predict(X_test_tfidf)
    model_c = LinearSVC()
    model_c.fit(X_train_cv,Y_train)
    Y_pred_cv = model_c.predict(X_test_cv)
    print("Individual class accuracies for Linear SVC  (tfidf vectiorizer)")
    print(classification_report(Y_test,Y_pred_t,target_names = [class_mapping[lbl],'not-'+class_mapping[lbl]]))
    print("Individual class accuracies for Linear SVC  (count vectiorizer)")
    print(classification_report(Y_test,Y_pred_cv,target_names = [class_mapping[lbl],'not-'+class_mapping[lbl]]))

Individual class accuracies for Linear SVC  (tfidf vectiorizer)
              precision    recall  f1-score   support

       anger       0.73      0.90      0.81        84
   not-anger       0.97      0.89      0.93       263

    accuracy                           0.90       347
   macro avg       0.85      0.90      0.87       347
weighted avg       0.91      0.90      0.90       347

Individual class accuracies for Linear SVC  (count vectiorizer)
              precision    recall  f1-score   support

       anger       0.72      0.90      0.80        84
   not-anger       0.97      0.89      0.93       263

    accuracy                           0.89       347
   macro avg       0.85      0.90      0.87       347
weighted avg       0.91      0.89      0.90       347

Individual class accuracies for Linear SVC  (tfidf vectiorizer)
              precision    recall  f1-score   support

        fear       0.81      0.80      0.81       110
    not-fear       0.91      0.92      0.91  

In [42]:
for lbl in [0,1,2,3]:
    
    train_0 = train[train['sentiment'] == lbl]
    train_rest = train[train['sentiment'] != lbl]
    train_rest = train_rest.sample(int(len(train_0)))
                                         

    train_sampled = train_0.append(train_rest)
    train_sampled = train_sampled.sample(frac=1).reset_index()

    Y_train = train_sampled['sentiment'].apply(lambda x: 0 if x == lbl else 1 ).tolist()
    Y_test = dev['sentiment'].apply(lambda x: 0 if x == lbl else 1 ).tolist()

    num_features = 3000
    model_tfidf = TfidfVectorizer(max_features=num_features)
    model_tfidf.fit(train['tweet'])

    model_cv = CountVectorizer(max_features=num_features)
    model_cv.fit(train['tweet'])

    X_train_tfidf = model_tfidf.transform(train_sampled['tweet']).toarray()
    X_test_tfidf = model_tfidf.transform(dev['tweet']).toarray()
    X_train_cv = model_cv.transform(train_sampled['tweet']).toarray()
    X_test_cv = model_cv.transform(dev['tweet']).toarray()
    

    model_t = RandomForestClassifier()
    model_t.fit(X_train_tfidf,Y_train)
    Y_pred_t = model_t.predict(X_test_tfidf)
    model_c = RandomForestClassifier()
    model_c.fit(X_train_cv,Y_train)
    Y_pred_cv = model_c.predict(X_test_cv)
    print("Individual class accuracies for Random Forest  (tfidf vectiorizer)")
    print(classification_report(Y_test,Y_pred_t,target_names = [class_mapping[lbl],'not-'+class_mapping[lbl]]))
    print("Individual class accuracies for Random Forest  (count vectiorizer)")
    print(classification_report(Y_test,Y_pred_cv,target_names = [class_mapping[lbl],'not-'+class_mapping[lbl]]))

Individual class accuracies for Random Forest  (tfidf vectiorizer)
              precision    recall  f1-score   support

       anger       0.76      0.85      0.80        84
   not-anger       0.95      0.91      0.93       263

    accuracy                           0.90       347
   macro avg       0.85      0.88      0.86       347
weighted avg       0.90      0.90      0.90       347

Individual class accuracies for Random Forest  (count vectiorizer)
              precision    recall  f1-score   support

       anger       0.85      0.85      0.85        84
   not-anger       0.95      0.95      0.95       263

    accuracy                           0.93       347
   macro avg       0.90      0.90      0.90       347
weighted avg       0.93      0.93      0.93       347

Individual class accuracies for Random Forest  (tfidf vectiorizer)
              precision    recall  f1-score   support

        fear       0.87      0.72      0.79       110
    not-fear       0.88      0.95   

In [43]:
for lbl in [0,1,2,3]:
    
    train_0 = train[train['sentiment'] == lbl]
    train_rest = train[train['sentiment'] != lbl]
    train_rest = train_rest.sample(int(len(train_0)))
                                         

    train_sampled = train_0.append(train_rest)
    train_sampled = train_sampled.sample(frac=1).reset_index()

    Y_train = train_sampled['sentiment'].apply(lambda x: 0 if x == lbl else 1 ).tolist()
    Y_test = dev['sentiment'].apply(lambda x: 0 if x == lbl else 1 ).tolist()

    num_features = 3000
    model_tfidf = TfidfVectorizer(max_features=num_features)
    model_tfidf.fit(train['tweet'])

    model_cv = CountVectorizer(max_features=num_features)
    model_cv.fit(train['tweet'])

    X_train_tfidf = model_tfidf.transform(train_sampled['tweet']).toarray()
    X_test_tfidf = model_tfidf.transform(dev['tweet']).toarray()
    X_train_cv = model_cv.transform(train_sampled['tweet']).toarray()
    X_test_cv = model_cv.transform(dev['tweet']).toarray()
    

    model_t = MLPClassifier()
    model_t.fit(X_train_tfidf,Y_train)
    Y_pred_t = model_t.predict(X_test_tfidf)
    model_c = MLPClassifier()
    model_c.fit(X_train_cv,Y_train)
    Y_pred_cv = model_c.predict(X_test_cv)
    print("Individual class accuracies for MLP Classifier  (tfidf vectiorizer)")
    print(classification_report(Y_test,Y_pred_t,target_names = [class_mapping[lbl],'not-'+class_mapping[lbl]]))
    print("Individual class accuracies for MLP Classifier  (count vectiorizer)")
    print(classification_report(Y_test,Y_pred_cv,target_names = [class_mapping[lbl],'not-'+class_mapping[lbl]]))

Individual class accuracies for MLP Classifier  (tfidf vectiorizer)
              precision    recall  f1-score   support

       anger       0.64      0.88      0.74        84
   not-anger       0.96      0.84      0.89       263

    accuracy                           0.85       347
   macro avg       0.80      0.86      0.82       347
weighted avg       0.88      0.85      0.86       347

Individual class accuracies for MLP Classifier  (count vectiorizer)
              precision    recall  f1-score   support

       anger       0.68      0.90      0.78        84
   not-anger       0.97      0.86      0.91       263

    accuracy                           0.87       347
   macro avg       0.82      0.88      0.84       347
weighted avg       0.90      0.87      0.88       347

Individual class accuracies for MLP Classifier  (tfidf vectiorizer)
              precision    recall  f1-score   support

        fear       0.69      0.75      0.72       110
    not-fear       0.88      0.85